In [12]:
import gensim
import nltk
from gensim import corpora,models, similarities
import json
import numpy as np
import keras
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import normalize
import re

Using TensorFlow backend.


In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('/media/rootmedia/modules/GoogleNews-vectors-negative300.bin/data', binary=True)

In [35]:
data = open("conversation.json")
data = json.load(data)
data = data['conversations']

x = []; y=[]


for i in range(len(data)):
    for j in range(len(data[i])):
        if j < len(data[i])-1:
            x.append(str(data[i][j]))
            y.append(str(data[i][j+1]))



In [54]:
sent_end = np.ones((300,), dtype=np.float32)

In [36]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [39]:
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]", " ", raw)
    words = clean.split()
    return words

In [40]:
token_x = tokenizer.tokenize(str(x))
token_y = tokenizer.tokenize(str(y))

sent_x = []
sent_y = []
for words in token_x:
    if len(words) > 0:
        sent_x.append(sentence_to_wordlist(words))

for wordy in token_y:
    if len(wordy) > 0:
        sent_y.append(sentence_to_wordlist(wordy))    

In [71]:
questions_x = []
answers_y = []

for sentence in sent_x:
    sent_vec = [model[w] for w in sentence if w in model.vocab]
    questions_x.append(sent_vec)
    pass

for answers in sent_y:
    sent_vec = [model[w] for w in answers if w in model.vocab]
    answers_y.append(sent_vec)

In [72]:
for tok_sent in questions_x:
    tok_sent[4:]=[]
    tok_sent.append(sent_end)
    
for tok_sent in answers_y:
    tok_sent[4:]=[]
    tok_sent.append(sent_end) 

In [73]:
train_x = np.array(questions_x[:70])
train_y = np.array(answers_y[:70])

In [78]:
print(np.array(train_x[7]).shape)
lenth = 300
shape = np.array(train_x[7]).shape

(5, 300)


In [80]:
chat = Sequential()

chat.add(LSTM(output_dim=lenth,input_shape=shape, return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
chat.add(LSTM(output_dim=lenth,input_shape=shape, return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
chat.add(LSTM(output_dim=lenth,input_shape=shape, return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
chat.add(LSTM(output_dim=lenth,input_shape=shape, return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))

chat.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

chat.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(5, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(5, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(5, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  """
/usr/local/lib/python3.6/dist

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 5, 300)            721200    
_________________________________________________________________
lstm_9 (LSTM)                (None, 5, 300)            721200    
_________________________________________________________________
lstm_10 (LSTM)               (None, 5, 300)            721200    
_________________________________________________________________
lstm_11 (LSTM)               (None, 5, 300)            721200    
Total params: 2,884,800
Trainable params: 2,884,800
Non-trainable params: 0
_________________________________________________________________


In [81]:
chat.fit(train_x, train_y, epochs=100, batch_size=10, verbose=2)

ValueError: Error when checking input: expected lstm_8_input to have 3 dimensions, but got array with shape (70, 1)